<a href="https://colab.research.google.com/github/CodeVins/Data-Analyze/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_%EA%B8%B0%EB%A7%90_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%88%98%EC%A7%91_%EA%B4%80%EB%A0%A8_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd

serviceKey = 'uun2aJj2%2Betud4g42QxMyfPiNHjo6GgWZ%2BTbaBmuCcIU4L3FE%2BKAFN%2Ff6POBGp4LPtScZQkt%2BZ6UuFefhcf5aA%3D%3D'
numOfRows = 400
startDt = 20130101
endDt = 20131231
stnId = 108  # 서울

def set_url(key, numOfRows, startDt, endDt, stnId, ctype):
    base = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList"
    url = (f"{base}?serviceKey={key}&pageNo=1&numOfRows={numOfRows}"
           f"&dataType={ctype}&dataCd=ASOS&dateCd=DAY"
           f"&startDt={startDt}&endDt={endDt}&stnIds={stnId}")
    return url

In [ ]:
url = set_url(serviceKey, numOfRows, startDt, endDt, stnId, "JSON")


response = requests.get(url)


if response.status_code == 200:
    data = response.json()
    items = data['response']['body']['items']['item']
    df = pd.DataFrame(items)
    print(" 데이터프레임 생성 완료")
else:
    print( response.status_code)
    print(response.text[:300])


 데이터프레임 생성 완료


In [ ]:
df['tm'] = pd.to_datetime(df['tm'])


df['month'] = df['tm'].dt.to_period('M')


df['sumRn'] = pd.to_numeric(df['sumRn'], errors='coerce').fillna(0)


monthly_summary = df.groupby('month').agg(
    월강수합계=('sumRn', 'sum'),
    월평균강수=('sumRn', 'mean'),
    비온날수=('sumRn', lambda x: (x > 0).sum()),
    전체일수=('sumRn', 'count')
).reset_index()


monthly_summary['비온날비율(%)'] = (monthly_summary['비온날수'] / monthly_summary['전체일수']) * 100


monthly_summary

monthly_summary.to_csv("seoul_2013_monthly_rainfall.csv", index=False, encoding='utf-8-sig')
print("'seoul_2013_monthly_rainfall.csv' 저장 완료!")


'seoul_2013_monthly_rainfall.csv' 저장 완료!


In [ ]:
url = 'https://raw.githubusercontent.com/CodeVins/seoul2013_violent_crime/main/seoul_2013_violent_crime.csv'
df = pd.read_csv(url, encoding='cp949')

df.head()

,월,서울 강력범죄 발생수
0,2013. 12,472
1,2013. 11,415
2,2013. 10,669
3,2013. 09,547
4,2013. 08,615


In [ ]:
file_path = '/content/seoul_2013_violent_crime.csv'

df.to_csv(file_path, index=False, encoding='utf-8-sig')

print(f"파일이 저장되었습니다: {file_path}")

파일이 저장되었습니다: /content/seoul_2013_violent_crime.csv


In [ ]:
rain_df = pd.read_csv('/content/seoul_2013_monthly_rainfall.csv', encoding='utf-8-sig')
crime_df = pd.read_csv('/content/seoul_2013_violent_crime.csv', encoding='utf-8-sig')

print("강수량 데이터 컬럼:", rain_df.columns.tolist())
print("범죄 데이터 컬럼:", crime_df.columns.tolist())

강수량 데이터 컬럼: ['month', '월강수합계', '월평균강수', '비온날수', '전체일수', '비온날비율(%)']
범죄 데이터 컬럼: ['월', '서울 강력범죄 발생수']


In [ ]:
crime_df = pd.read_csv('/content/seoul_2013_violent_crime.csv', encoding='utf-8-sig')


crime_df['month'] = pd.to_datetime(crime_df['월'], format='%Y. %m').dt.to_period('M')


crime_df = crime_df.drop(columns=['월'])


crime_df.to_csv('seoul_2013_monthly_violent_crime_preprocessed.csv', index=False, encoding='utf-8-sig')
print(" 저장 완료.")


 저장 완료.


In [ ]:
rain_df = pd.read_csv('/content/seoul_2013_monthly_rainfall.csv', encoding='utf-8-sig')
crime_df = pd.read_csv('/content/seoul_2013_monthly_violent_crime_preprocessed.csv', encoding='utf-8-sig')

rain_df['month'] = pd.to_datetime(rain_df['month']).dt.to_period('M')
crime_df['month'] = pd.to_datetime(crime_df['month']).dt.to_period('M')

merged_df = pd.merge(rain_df, crime_df, on='month', how='inner')

print(merged_df)

merged_df.to_csv('seoul_2013_rainfall_violent_crime_merged.csv', index=False, encoding='utf-8-sig')


      month  월강수합계      월평균강수  비온날수  전체일수   비온날비율(%)  서울 강력범죄 발생수
0   2013-01   22.1   0.712903     7    31  22.580645          396
1   2013-02   74.1   2.646429     8    28  28.571429          346
2   2013-03   27.3   0.880645     8    31  25.806452          444
3   2013-04   71.7   2.390000    15    30  50.000000          546
4   2013-05  132.0   4.258065    10    31  32.258065          558
5   2013-06   28.3   0.943333     9    30  30.000000          619
6   2013-07  676.2  21.812903    25    31  80.645161          649
7   2013-08  148.6   4.793548    12    31  38.709677          615
8   2013-09  138.5   4.616667     9    30  30.000000          547
9   2013-10   13.5   0.435484     4    31  12.903226          669
10  2013-11   46.8   1.560000    12    30  40.000000          415
11  2013-12   24.7   0.796774     9    31  29.032258          472
